# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [68]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core import Workspace, Experiment
from azureml.widgets import RunDetails
import joblib
from azureml.core import Environment
from azureml.core.model import InferenceConfig,Model
from azureml.core.webservice import AciWebservice,Webservice,LocalWebservice
from azureml.train.automl import AutoMLConfig
from azureml.core.compute_target import ComputeTargetException
import requests
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import math

## Dataset

### Overview
The uploaded dataset is a bond data. The targeted variable is OAS - Option Adjusted Spread. This spread is a quantitative representation of the risk of the bond. It depends on many factors, one of the most important of which is the bonds's rating. The variables representing the rating is Warf (numerical format). Additionally, the dataset contains various indicators such as if the bond belongs to a certain type of industry. The goal is to predict the OAS using the input variables. Since it is a numeric continious variable, a regression will be chosen.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

In [3]:
df_bonds = ws.datasets['df_bonds'] 
experiment=Experiment(ws, 'bonds_oas')
cluster_name='valeriy-cluster'
try:
    cluster = ComputeTarget(workspace=ws, name=cluster_name)
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',max_nodes=4)
    cluster = ComputeTarget.create(ws, cluster_name, compute_config)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [6]:
# TODO: Put your automl settings here
automl_settings = {
"experiment_timeout_minutes": 15,
"max_concurrent_iterations":20,
'n_cross_validations':2,
"primary_metric" : 'r2_score'
}
project_folder = './bonds-project'
automl_config = AutoMLConfig(compute_target=cluster,
                             task = "regression",
                             training_data=df_bonds,
                             label_column_name="OAS",   
                             path = project_folder,
                             debug_log = "automl_errors.log",                            
                             **automl_settings
                            )

In [7]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)
remote_run.wait_for_completion()

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
bonds_oas,AutoML_ed3676e2-c0c7-4066-8772-69c21d2030ac,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


{'runId': 'AutoML_ed3676e2-c0c7-4066-8772-69c21d2030ac',
 'target': 'valeriy-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-02-20T11:06:58.83504Z',
 'endTimeUtc': '2022-02-20T11:25:19.854261Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'Experiment timeout reached, hence experiment stopped. Current experiment timeout: 0 hour(s) 15 minute(s)'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'r2_score',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '2',
  'target': 'valeriy-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"bonds_oas","subscription_id":"6971f5ac-8af1-446e-8034-05acea24681f","resource_group":"aml-quickstarts-187155","workspace_name":"quick-starts-ws-187155","region":"westeurope","compute_target":"valeriy-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [73]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [9]:
best_run_automl, fitted_model_automl=remote_run.get_output()
model_name=best_run_automl.properties['model_name']
fitted_model_automl.steps

[('datatransformer',
  DataTransformer(
      task='regression',
      is_onnx_compatible=False,
      enable_feature_sweeping=True,
      enable_dnn=False,
      force_text_dnn=False,
      feature_sweeping_timeout=86400,
      featurization_config=None,
      is_cross_validation=True,
      feature_sweeping_config={}
  )),
 ('prefittedsoftvotingregressor',
  PreFittedSoftVotingRegressor(
      estimators=[('1', Pipeline(
          memory=None,
          steps=[('maxabsscaler', MaxAbsScaler(
              copy=True
          )), ('xgboostregressor', XGBoostRegressor(
              random_state=0,
              n_jobs=1,
              problem_info=ProblemInfo(
                  gpu_training_param_dict={'processing_unit_type': 'cpu'}
              ),
              tree_method='auto'
          ))],
          verbose=False
      )), ('18', Pipeline(
          memory=None,
          steps=[('standardscalerwrapper', StandardScalerWrapper(
              copy=True,
              with_mean=Fal

In [27]:
#TODO: Save the best model
joblib.dump(fitted_model_automl, 'model.pkl')
joblib.dump(fitted_model_automl, 'inference/model.pkl')

['inference/model.pkl']

In [13]:
#Check the performance of the best model. This is required in order to compare it with the hyperdrive one
df_bonds = ws.datasets['df_bonds']
x=df_bonds.to_pandas_dataframe().drop('OAS',1)
y=df_bonds.to_pandas_dataframe()['OAS']
   
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.30,random_state=10)
y_pred_automl = fitted_model_automl.predict(x_test)
rmse_automl = math.sqrt(mean_squared_error(y_true=y_test, y_pred=y_pred_automl))
print('automl:',rmse_automl)

automl: 271.9051570541046


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
#Register model
description = 'Bonds data'
model = remote_run.register_model(model_name=model_name,
                          description=description,
                           tags=None)
print(remote_run.model_id)

In [42]:
#Create an inference configuration:
env = Environment(name="valeriy_env")
from azureml.core.conda_dependencies import CondaDependencies
script_file_name='inference/score_bonds.py'
best_run_automl.download_file('outputs/scoring_file_v_1_0_0.py',script_file_name)
inference_config = InferenceConfig(entry_script=script_file_name,environment=best_run_automl.get_environment())
aciconfig=AciWebservice.deploy_configuration(cpu_cores=1,memory_gb=1,tags={'type':'bonds_df'}, description='n')
aci_service=Model.deploy(ws,'valeriywebservice7',[model],inference_config,aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-02-20 13:22:25+00:00 Creating Container Registry if not exists.
2022-02-20 13:22:26+00:00 Registering the environment.
2022-02-20 13:22:26+00:00 Use the existing image.
2022-02-20 13:22:27+00:00 Generating deployment configuration.
2022-02-20 13:22:28+00:00 Submitting deployment to compute.
2022-02-20 13:22:32+00:00 Checking the status of deployment valeriywebservice7..
2022-02-20 13:26:29+00:00 Checking the status of inference endpoint valeriywebservice7.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


TODO: In the cell below, send a request to the web service you deployed to test it.

In [60]:
import json
data_to_predict=x_test.iloc[:1]
test_sample=json.dumps({'data':data_to_predict.to_dict(orient='records')})
response = aci_service.run(input_data=test_sample)
print(response)

{"result": [337.6384513314485]}


TODO: In the cell below, print the logs of the web service and delete the service

In [61]:
#Save the enviroment details
env_used=best_run_automl.get_environment()
env_used.save_to_directory('automl_env.txt', overwrite=True)

In [ ]:


print(aci_service.get_logs())
aci_service.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
